In [ ]:
import os
import json
import pandas as pd
import numpy as np
import nltk
import gensim
import re
import torch
import torchvision
from gensim.models import Word2Vec
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig, TrainingArguments, Trainer, EarlyStoppingCallback
from datasets import load_dataset
from collections import Counter
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

class WeightedLossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

dataset = "chengxuphd/liar2"
dataset = load_dataset(dataset)

pretrained_model = "roberta-base"

tokenizer = AutoTokenizer.from_pretrained(pretrained_model)

def preprocess_function(examples):
    combined_input = [
        "Subject: " + (subject if subject is not None else "") + 
        "; Speaker: " + (speaker if speaker is not None else "") + 
        "; Speaker Description: " + (speaker_description if speaker_description is not None else "") + 
        "; State: " + (state_info if state_info is not None else "") + 
        "; Context: " + (context if context is not None else "") + 
        "; Statement: " + (statement if statement is not None else "")
        for subject, speaker, speaker_description, state_info, context, statement in zip(
            examples["subject"],
            examples["speaker"],
            examples["speaker_description"],
            examples["state_info"],
            examples["context"],
            examples["statement"]
        )
    ]
    return tokenizer(combined_input, padding="max_length", truncation=True)

tokenized_datasets = dataset.map(preprocess_function, batched=True)

tokenized_datasets = tokenized_datasets.remove_columns(["id", "subject", "speaker", "speaker_description", "state_info", "context", "true_counts", "mostly_true_counts", "half_true_counts", "mostly_false_counts", "false_counts", "pants_on_fire_counts", "justification"])

label_to_binary = {
    # label_to_id = {"REAL": 0, "FAKE": 1}
    0: True,
    1: True,
    2: True,
    3: True, # Changed to FAKE
    4: False,
    5: False
}

tokenized_datasets = tokenized_datasets.map(
    lambda examples: {"label": [label_to_binary[int(label)] for label in examples["label"]]},
    batched=True
)

tokenized_datasets.set_format("torch")

train_dataset = tokenized_datasets["train"]
val_dataset = tokenized_datasets["validation"]
test_dataset = tokenized_datasets["test"]

# Calculate normalized class weights
count0 = sum(1 for label in train_dataset["label"] if label == 0)
count1 = sum(1 for label in train_dataset["label"] if label == 1)
total = count0 + count1
n_classes = 2
weight_0 = total / (n_classes * count0)
weight_1 = total / (n_classes * count1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class_weights = torch.tensor([weight_0, weight_1]).to(device)

# Define compute_metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    acc = accuracy_score(labels, preds)
    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall,
    }

# Define training arguments (epoch)
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    learning_rate=2e-5,
    lr_scheduler_type="cosine",
    warmup_steps=500,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none",
    load_best_model_at_end=True,  # Load the best model at the end of training
    metric_for_best_model="recall",  # Specify the metric to monitor
    greater_is_better=True       # Specify if higher values of the metric are better
)

config = AutoConfig.from_pretrained(pretrained_model, num_labels=2, hidden_dropout_prob=0.2, attention_probs_dropout_prob=0.2)
model = AutoModelForSequenceClassification.from_pretrained(pretrained_model, config=config)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model)
model = model.to(device)

# Swap the labels in the model configuration
model.config.id2label = {0: "REAL", 1: "FAKE"}
model.config.label2id = {"REAL": 0, "FAKE": 1}

# Verify the changes
print("Updated id2label mapping:", model.config.id2label)
print("Updated label2id mapping:", model.config.label2id)

training_args.num_train_epochs = 5
training_args.learning_rate = 2e-5
print("Model is on:", next(model.parameters()).device)
print("Learning rate:", training_args.learning_rate)

trainer = WeightedLossTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

trainer.train()

In [ ]:
trainer.train(resume_from_checkpoint="/kaggle/working/results/checkpoint-1725")

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("/kaggle/working/results/checkpoint-1725")
trainer.model = model
trainer.model.to(device)

In [ ]:
trainer.evaluate()

In [ ]:
# Conduct testing on the test dataset
test_results = trainer.predict(test_dataset)

# Extract predictions and metrics
predictions = test_results.predictions.argmax(-1)  # Convert logits to class predictions
metrics = test_results.metrics  # Contains accuracy, F1, precision, recall, etc.

# Print metrics
print("Test Metrics:")
for key, value in metrics.items():
    print(f"{key}: {value:.4f}")

In [ ]:
trainer.save_model("./models/weighted_loss/roberta")